- # Integração Arduino com Python
***

- #### Add from Arduino import Arduino into your python script to communicate with your Arduino.

In [3]:
from Arduino import Arduino
import time
import json
import requests
from datetime import datetime

***
- ## Definindo variáveis

In [4]:
green_led, red_led = 13, 7
detector_pin = 3

***
- ## void setup()

In [5]:
board = Arduino("9600", port="/dev/ttyACM1") # plugged in via USB, serial com at rate 9600

board.pinMode(detector_pin, "INPUT") # Configures DETECTOR pin as input
board.pinMode(green_led, "OUTPUT") # Configures LED pin as output
board.pinMode(red_led, "OUTPUT") # Configures LED pin as output

You need to update the version of the Arduino-Python3 library running on your Arduino.
The Arduino sketch is version
The Python installation is V0.6
Flash the prototype sketch again.


***

- ## void loop()

In [6]:
URL = 'http://127.0.0.1:5000/'

In [10]:
while True:

    if (board.digitalRead(detector_pin)):
        board.digitalWrite(green_led, "LOW")
        board.digitalWrite(red_led, "HIGH")

        body_presence = {
            "date_time_sent": str(datetime.today()),
            "sensor_id": "AUPS1",
            "detection": 1
        }
        post = requests.request('POST', URL + '/presence', json=body_presence)
        print(post)
        time.sleep(2)

    else:
        board.digitalWrite(red_led, "LOW")
        board.digitalWrite(green_led, "HIGH")
        body_presence = {
            "date_time_sent": str(datetime.today()),
            "sensor_id": "AUPS1",
            "detection": 0
        }
        post = requests.request('POST', URL + '/presence', json=body_presence)
        print(post)
        time.sleep(2)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


KeyboardInterrupt: 

In [11]:
delete_data_from_table = requests.request('DELETE', URL + '/presences')
delete_data_from_table

<Response [200]>